In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from datetime import date, datetime, timedelta

In [2]:
datos = pd.read_csv('Casos_Diarios_Municipio_Confirmados_20201029.csv')
datos['cve_ent'] = [f'{x:05}' for x in datos['cve_ent']]

In [3]:
datos.head()

,cve_ent,poblacion,nombre,12-01-2020,13-01-2020,14-01-2020,15-01-2020,16-01-2020,17-01-2020,18-01-2020,...,20-10-2020,21-10-2020,22-10-2020,23-10-2020,24-10-2020,25-10-2020,26-10-2020,27-10-2020,28-10-2020,29-10-2020
0,16046,15290,Juarez,0,0,0,0,0,0,0,...,2,0,0,0,2,0,0,0,0,0
1,16047,22358,Jungapeo,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,0,0
2,16048,5862,Lagunillas,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,16049,18769,Madero,0,0,0,0,0,0,0,...,0,1,2,0,0,0,0,0,0,0
4,05035,744247,Torreon,0,0,0,0,0,0,0,...,122,133,90,102,88,52,41,26,9,0


In [4]:
IDS = pd.DataFrame()
IDS['CVEGEO'] = datos['cve_ent']

In [5]:
datos = datos.drop(columns=['cve_ent', 'poblacion', 'nombre'])
datos.columns = pd.to_datetime(datos.columns, format='%d-%m-%Y')
datos = datos.cumsum(axis=1)
datos.head()

,2020-01-12,2020-01-13,2020-01-14,2020-01-15,2020-01-16,2020-01-17,2020-01-18,2020-01-19,2020-01-20,2020-01-21,...,2020-10-20,2020-10-21,2020-10-22,2020-10-23,2020-10-24,2020-10-25,2020-10-26,2020-10-27,2020-10-28,2020-10-29
0,0,0,0,0,0,0,0,0,0,0,...,73,73,73,73,75,75,75,75,75,75
1,0,0,0,0,0,0,0,0,0,0,...,28,30,30,30,30,30,30,30,30,30
2,0,0,0,0,0,0,0,0,0,0,...,29,29,29,29,29,29,29,29,29,29
3,0,0,0,0,0,0,0,0,0,0,...,40,41,43,43,43,43,43,43,43,43
4,0,0,0,0,0,0,0,0,0,0,...,7063,7196,7286,7388,7476,7528,7569,7595,7604,7604


In [6]:
#crea una lista con los indices separados en un intervalo de siete dias
def sepsemanas(start, end):
    semana = start
    l = [semana]
    while semana < end:
        l.append(semana)
        semana = semana + timedelta(days = 7)
    else:
        l.append(end)
    return l

semanas = sepsemanas(datos.columns[0], datos.columns[-1])

In [7]:
c = 1

for i  in semanas:

    col = i
    
    confirmados = datos[col].sum()
    fecha = str(col.date())
    
    temp = IDS.join(datos[col])
    temp = temp.rename(columns={col: fecha})
    
    mapa = gpd.read_file('DIRECCION/MapaMéxicoMunicipal/mun2019gw.shp')
    mapatemp = pd.merge(mapa, temp, on=['CVEGEO'])
    mapatemp['test'] = mapatemp[fecha]
    mapatemp[fecha] = mapatemp[fecha].fillna(0)
    
    fig, ax = plt.subplots(1, 1)
    plt.title('Casos confirmados por municipio para la fecha ' + fecha)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.1)
    mapatemp.plot(column=fecha, ax=ax, legend=True, figsize=(1000, 970), cax=cax)
    ax.annotate("Número de casos confirmados totales: " + str(confirmados), xy=(1, 0), xycoords='axes fraction', fontsize=11,
                   xytext=(-5, 5), textcoords='offset points',
                   ha='right', va='bottom')
    plt.savefig('casosconf' + str(c) + '.png', dpi=500)
    plt.clf()
    c = c + 1

<ipython-input-7-cd4a16fa007d>:18: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(1, 1)
<ipython-input-7-cd4a16fa007d>:18: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(1, 1)
<ipython-input-7-cd4a16fa007d>:18: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(1

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

...

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>